In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# Leemos los archivos CSV
from pyspark.sql.types import StructField, StructType, StringType, DateType, IntegerType, DoubleType

memberSchema = StructType([
    StructField("customer_id_members", StringType(), False),
    StructField("join_date", DateType(), False),
])

membersDF = (spark
             .read
             .option("header",True)
             .schema(memberSchema)
             .csv("/FileStore/tables/members.csv")
 
)

menuSchema = StructType([
    StructField("product_id_menu", IntegerType(), False),
    StructField("product_name", StringType(), False),
    StructField("price", DoubleType(), False),
])

menuDF = (spark
             .read
             .option("header",True)
             .schema(menuSchema)
             .csv("/FileStore/tables/menu.csv")
 
)

salesSchema = StructType([
    StructField("customer_id_sales", StringType(), False),
    StructField("order_date", DateType(), False),
    StructField("product_id_sales", IntegerType(), False),
])

salesDF = (spark
             .read
             .option("header",True)
             .schema(salesSchema)
             .csv("/FileStore/tables/sales.csv")
 
)
display(membersDF)


customer_id_members,join_date
A,2021-01-07
B,2021-01-09


In [0]:
from pyspark.sql.functions import col
joinType = "inner"
joinExpression = membersDF["customer_id_members"] == salesDF ['customer_id_sales']
joinDF1 = salesDF.join(membersDF,joinExpression,joinType)


joinExpression2 = joinDF1["product_id_sales"] == menuDF ['product_id_menu']
joinDF2 = joinDF1.join(menuDF,joinExpression2,joinType)

display(joinDF2)


customer_id_sales,order_date,product_id_sales,customer_id_members,join_date,product_id_menu,product_name,price
A,2021-01-01,1,A,2021-01-07,1,sushi,10.0
A,2021-01-01,2,A,2021-01-07,2,curry,15.0
A,2021-01-07,2,A,2021-01-07,2,curry,15.0
A,2021-01-10,3,A,2021-01-07,3,ramen,12.0
A,2021-01-11,3,A,2021-01-07,3,ramen,12.0
A,2021-01-11,3,A,2021-01-07,3,ramen,12.0
B,2021-01-01,2,B,2021-01-09,2,curry,15.0
B,2021-01-02,2,B,2021-01-09,2,curry,15.0
B,2021-01-04,1,B,2021-01-09,1,sushi,10.0
B,2021-01-11,1,B,2021-01-09,1,sushi,10.0


In [0]:
#Pregunta1: ¿Cuál es la cantidad total que gastó cada cliente en el restaurante?
from pyspark.sql.functions import col,sum,desc

display(joinDF2.groupBy(col("customer_id_sales").alias("Cliente")).agg(sum("price").alias("Total_Gastado")).orderBy(col("Total_Gastado").desc()))


Cliente,Total_Gastado
A,76.0
B,74.0


In [0]:
#Pregunta2: ¿Cuántos días ha visitado cada cliente el restaurante?
from pyspark.sql.functions import col,countDistinct,desc

display(joinDF2.groupBy(col("customer_id_sales").alias("Cliente")).agg(countDistinct("order_date").alias("Dias")).orderBy(col("Dias").desc()))


Cliente,Dias
B,6
A,4


In [0]:
#Pregunta3: ¿Cuál fue el primer artículo del menú comprado por cada cliente?

joinDF2.createOrReplaceTempView("sqlDF")


In [0]:
%sql

SELECT A.customer_id_sales,product_name FROM (
SELECT 
  customer_id_sales,
  --product_name,
  MIN(order_date) AS date

FROM sqlDF
GROUP BY customer_id_sales--,product_name
--HAVING MIN(order_date)
) AS A
INNER JOIN sqlDF AS B ON A.customer_id_sales = B. customer_id_sales AND A.date = B.order_date


customer_id_sales,product_name
B,curry
A,curry
A,sushi


In [0]:
#Pregunta4: ¿Cuál es el artículo más comprado en el menú y cuántas veces lo compraron todos los clientes?
from pyspark.sql.functions import count,col

display(joinDF2.groupBy(col("product_name")).agg(count("product_id_sales").alias("Cantidad")).orderBy(col("Cantidad").desc()).take(1))


product_name,Cantidad
ramen,5


In [0]:
#Pregunta5: ¿Qué artículo fue el más popular para cada cliente?

from pyspark.sql.functions import count,col,max

artDF = (joinDF2.groupBy(col("customer_id_sales"),col("product_name")).agg(count("product_id_sales").alias("Cantidad")).orderBy(col("Cantidad").desc()))

maxArtDF = artDF.groupBy(col("customer_id_sales")).agg(max("Cantidad").alias("MaxCant"))
#maxArtDF.show()

joinType = "inner"
joinExpr = artDF["customer_id_sales"] == maxArtDF ['customer_id_sales']
maxArtDF2 = artDF.join(maxArtDF,joinExpr,joinType)
display(maxArtDF2.where("Cantidad = MaxCant"))




customer_id_sales,product_name,Cantidad,customer_id_sales,MaxCant
A,ramen,3,A,3
B,sushi,2,B,2
B,ramen,2,B,2
B,curry,2,B,2


In [0]:
# Pregunta6: ¿Qué artículo compró primero el cliente después de convertirse en miembro?

from pyspark.sql.functions import min as min_

#df = (joinDF2.withColumn("order_date_2", col("order_date").cast("timestamp")))
df = joinDF2.where("order_date > join_date").groupBy("customer_id_sales","product_name").agg(min_("order_date").alias("date"))
#df.show()
df2 = joinDF2.where("order_date > join_date").groupBy("customer_id_sales").agg(min_("order_date").alias("date2"))

joinType = "inner"
joinExpr = df["customer_id_sales"] == df2 ['customer_id_sales']
df3 = df.join(df2,joinExpr,joinType)
display(df3.where("date = date2"))


customer_id_sales,product_name,date,customer_id_sales,date2
B,sushi,2021-01-11,B,2021-01-11
A,ramen,2021-01-10,A,2021-01-10


In [0]:
#Pregunta7: ¿Qué artículo se compró justo antes de que el cliente se convirtiera en miembro?

from pyspark.sql.functions import count,col,max as max_

#df = (joinDF2.withColumn("order_date_2", col("order_date").cast("timestamp")))
df = (joinDF2.where("order_date < join_date").groupBy("customer_id_sales").agg(max_("order_date").alias("date")))

df2 = (joinDF2.where("order_date < join_date").groupBy("customer_id_sales","product_name").agg(max_("order_date").alias("date2")))

joinType = "inner"
joinExpr = df["customer_id_sales"] == df2 ['customer_id_sales']
df3 = df.join(df2,joinExpr,joinType)
display(df3.where("date = date2"))



customer_id_sales,date,customer_id_sales,product_name,date2
A,2021-01-01,A,sushi,2021-01-01
B,2021-01-04,B,sushi,2021-01-04
A,2021-01-01,A,curry,2021-01-01


In [0]:
#Pregunta8: ¿Cuál es el total de artículos y la cantidad gastada por cada miembro antes de convertirse en miembro?
from pyspark.sql.functions import count,expr,col,sum 

display(joinDF2.where("order_date < join_date").groupBy("customer_id_sales").agg(count("product_id_sales").alias("Total Articulos"),sum("price").alias("Total Gastado")))




customer_id_sales,Total Articulos,Total Gastado
B,3,40.0
A,2,25.0


In [0]:
#Pregunta9: Si cada $ 1 gastado equivale a 10 puntos y el sushi tiene un multiplicador de puntos 2x, ¿cuántos puntos tendría cada cliente?
from pyspark.sql.functions import expr, col, sum
puntosDF = (joinDF2.withColumn("Puntos",expr("CASE WHEN product_name = 'sushi' THEN 10*price*2 ELSE 10*price END")))
puntosDF.groupBy("customer_id_sales").agg(sum("Puntos").alias("Total Puntos")).show()


+-----------------+------------+
|customer_id_sales|Total Puntos|
+-----------------+------------+
|                B|       940.0|
|                A|       860.0|
+-----------------+------------+



In [0]:
#Pregunta10: En la primera semana después de que un cliente se une al programa (incluida la fecha de ingreso), gana el doble de puntos en todos los artículos, no solo en sushi. ¿Cuántos puntos tienen los clientes A y B a fines de enero?
from pyspark.sql.functions import expr, col, sum

puntosDF2 = (joinDF2.withColumn("Puntos",expr("CASE WHEN order_date >= join_date AND order_date <= DATEADD(day, 7, join_date) THEN 10*price*2 ELSE 10*price END")))

display(puntosDF2.filter(col("order_date") <= '2021-01-31').groupBy("customer_id_sales").agg(sum("Puntos").alias("Total Puntos")))

customer_id_sales,Total Puntos
B,840.0
A,1270.0
